<a href="https://colab.research.google.com/github/benmsanderson/SRMfair/blob/main/FAIR_SRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
!pip install fair==1.6.4
import fair

In [ ]:
import numpy as np
from scipy import stats
from scipy import signal
from fair.tools.ensemble import tcrecs_generate
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn-muted')
plt.rcParams['figure.figsize'] = (16, 9)

In [ ]:
from fair.RCPs import rcp3pd, rcp45, rcp6, rcp85
from fair.SSPs import ssp370, ssp126, ssp585, ssp119,ssp245,ssp534, ssp460

In [ ]:
#start with test simulations built on SSP3-RCP7
ssp370.Emissions.year[257]
g10_rf=ssp370.Emissions.emissions[:,1]*0
g10_rf[257:287]=-60*np.exp(-np.arange(1,31,1)/6.3)
g5_rf=ssp370.Emissions.emissions[:,1]*0
g5_rf[257:287]=-27*np.exp(-np.arange(1,31,1)/6.3)
g1_rf=ssp370.Emissions.emissions[:,1]*0
g1_rf[257:287]=-6*np.exp(-np.arange(1,31,1)/6.3)

In [ ]:
#function to simulate nuclear pulse given initial forcing level and decay time
def nfn(a,t):
  ans=ssp370.Emissions.emissions[:,1]*0
  ans[257:307]=-a*np.exp(-np.arange(1,51,1)/t)
  return ans

In [ ]:
#function to simulate nuclear pulse given initial forcing level and decay time
def sfn(a,t):
  ans=ssp370.Emissions.emissions[:,1]*0
  ans[0:306]=-a*np.exp(-np.arange(1,307,1)/t)
  return ans

In [ ]:
len(ssp370.Emissions.emissions[:,1]*0)


In [ ]:
sfn(2,5).shape

In [ ]:

fig = plt.figure(figsize=(12, 11))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
#conventional SSP-RCPs
C26, F26, T26 = fair.forward.fair_scm(emissions=ssp126.Emissions.emissions)
ax1.plot(rcp3pd.Emissions.year, ssp126.Emissions.co2_fossil, color='green', label='SSP126')
ax2.plot(rcp3pd.Emissions.year, C26[:, 0], color='green')
ax3.plot(rcp3pd.Emissions.year, np.sum(F26, axis=1), color='green')
ax4.plot(rcp3pd.Emissions.year, T26, color='green')

C37, F37, T37 = fair.forward.fair_scm(emissions=ssp370.Emissions.emissions)
C45, F45, T45 = fair.forward.fair_scm(emissions=ssp245.Emissions.emissions)
ax1.plot(rcp3pd.Emissions.year, ssp370.Emissions.co2_fossil, color='black', label='SSP370')
ax2.plot(rcp3pd.Emissions.year, C37[:, 0], color='black')
ax3.plot(rcp3pd.Emissions.year, np.sum(F37, axis=1), color='black')
ax4.plot(rcp3pd.Emissions.year, T37, color='black')

ax1.plot(rcp3pd.Emissions.year, ssp370.Emissions.co2_fossil, color='black', label='SSP370')
ax2.plot(rcp3pd.Emissions.year, C37[:, 0], color='black')
ax3.plot(rcp3pd.Emissions.year, np.sum(F37, axis=1), color='black')
ax4.plot(rcp3pd.Emissions.year, T37, color='black')

C37g0, F37g0, T37g0 = fair.forward.fair_scm(emissions=np.sum(ssp370.Emissions.emissions[:,1:3],axis=1),other_rf=np.sum(F37[:,1:], axis=1),
    useMultigas=False)
ax1.plot(rcp3pd.Emissions.year, ssp370.Emissions.co2_fossil, color='y', label='SSP370_10G')
ax2.plot(rcp3pd.Emissions.year, C37g0, color='y',linestyle=':')
ax3.plot(rcp3pd.Emissions.year, F37g0, color='y',linestyle=':')
ax4.plot(rcp3pd.Emissions.year, T37g0, color='y',linestyle=':')

#for the nuclear simulations, we run with Mutligas off.  nonco2 radiative forcing from the baseline case is used, in addition to the nuclear pulse in the other_rf term
#the following forcings were munually adjusted to 65 and 30Wm^-2 to match the 2 and 8k cooling seen in Robock 2007


srmsig=np.hstack((np.zeros((256)),.2*np.ones((50)),np.zeros((430))))
srm_fcg=signal.convolve(srmsig,sfn(1,5),mode='full')[:736]

C37g150, F37g150, T37g150 = fair.forward.fair_scm(emissions=np.sum(ssp370.Emissions.emissions[:,1:3],axis=1),other_rf=srm_fcg+np.sum(F37[:,1:], axis=1),
    useMultigas=False)
ax1.plot(rcp3pd.Emissions.year, ssp370.Emissions.co2_fossil, color='darkred', label='SSP370_150')
ax2.plot(rcp3pd.Emissions.year, C37g150, color='darkred')
ax3.plot(rcp3pd.Emissions.year, F37g150, color='darkred')
ax4.plot(rcp3pd.Emissions.year, T37g150, color='darkred')


ax1.set_ylabel('Fossil CO$_2$ Emissions (GtC)')
ax1.legend()
ax2.set_ylabel('CO$_2$ concentrations (ppm)')
ax3.set_ylabel('Total radiative forcing (W.m$^{-2}$)')
ax4.set_ylabel('Temperature anomaly (K)');
ax1.set_xlim(1850,2100)
ax2.set_xlim(1850,2100)
ax3.set_xlim(1850,2100)
ax4.set_xlim(1850,2100)



In [ ]:
yrs=np.arange(2020,2300)
ems_bs=np.sum(ssp534.Emissions.emissions[:,1:3],axis=1)
C34, F34, T34 = fair.forward.fair_scm(emissions=ssp534.Emissions.emissions)

f_bs=np.sum(F34[:,1:11], axis=1)
yr=ssp370.Emissions.emissions[:,0]
C45g0, F45g0, T45g0 = fair.forward.fair_scm(emissions=ems_bs,other_rf=f_bs,useMultigas=False)

In [ ]:
def run_fair_clean(ems):
  Ce, Fe, Te = fair.forward.fair_scm(emissions=ems.Emissions.emissions)
  ems_bs=np.sum(ems.Emissions.emissions[:,1:3],axis=1)
  f_bs=np.sum(Fe[:,1:11], axis=1)
  C45g0, F45g0, T45g0 = fair.forward.fair_scm(emissions=ems_bs,other_rf=f_bs,useMultigas=False)
  return C45g0, F45g0, T45g0

In [ ]:
def adpt_fair(ems,sint,threshold,df,wd=2500,wf=1):
  Ce, Fe, Te = fair.forward.fair_scm(emissions=ems.Emissions.emissions)
  ems_bs=np.sum(ems.Emissions.emissions[:,1:3],axis=1)
  f_bs=np.sum(Fe[:,1:11], axis=1)
  C45g0, F45g0, T45g0 = fair.forward.fair_scm(emissions=ems_bs,other_rf=f_bs,useMultigas=False)

  C0, F0, T0 = fair.forward.fair_scm(emissions=ems_bs*0,other_rf=f_bs*0,useMultigas=False)

  Ttmp1=T45g0
  srm1=f_bs*0
  ems1=ems_bs.copy()
  for i in np.arange(0,60):
    Ctmp1, Ftmp1, Ttmp1 = fair.forward.fair_scm(emissions=ems1,other_rf=f_bs+srm1,useMultigas=False)
    ovsht=(Ttmp1-threshold).clip(min=0)
    srm1=srm1-ovsht/sint
  srm_out=srm1*0
  for i in np.arange(len(df)):
    istart=int(df.loc[i]['Start']-ems.Emissions.emissions[0,0])
    iend=int(df.loc[i]['End']-ems.Emissions.emissions[0,0])
    ifade=int(df.loc[i]['fade'])

    srm_out[istart:iend]=srm1[istart:iend]*df.loc[i].Effic
    if ifade>0:
      srm_out[iend:(iend+ifade)]=srm1[iend]*df.loc[i].Effic*(1-np.arange(0,ifade)/ifade)

  wi=int(wd-ssp126.Emissions.emissions[0,0])
  emsw=ems1.copy()
  emsw[wi:]=ems1[wi:]*wf
  Ctmp1, Ftmp1, Ttmp1 = fair.forward.fair_scm(emissions=emsw,other_rf=f_bs+srm_out,useMultigas=False)


  return Ctmp1, Ftmp1, Ttmp1, srm_out, emsw

In [ ]:
d = {'Start': [2025], 'End': [2150], 'Effic': [1.0], 'fade':[0]}
df = pd.DataFrame(data=d)
df

In [ ]:
ssp534.Emissions.emissions[0,0]

In [ ]:
Ctmp1s, Ftmp1s, Ttmp1s, srm1s, ems1s=adpt_fair(ssp534,10,1.5,df)

In [ ]:
gamma=-50
beta=0.8

In [ ]:
fig = plt.figure(figsize=(5, 8))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)


ax1.plot(yr,T34,'r',label='SSP534-over')
ax1.plot(yr,Ttmp1s,'y',label='SSP534-geo')
ax1.set_xlim([1990,2300])
ax1.legend()
ax1.set_ylabel('Warming (K)')

ax2.plot(yr,ems_bs,'k',label='Emissions (FF+LU)')
ax2.plot(yr,-gamma*np.diff(T45g0,prepend=0)-beta*np.diff(C45g0,prepend=C34[0,0]),'r',label='land+ocn sink (SSP534-over)')
ax2.plot(yr,-gamma*np.diff(Ttmp1s,prepend=0)-beta*np.diff(Ctmp1s,prepend=C34[0,0]),'y',label='land+ocn sink (SSP534-geo)')
ax2.set_ylabel('(GtC/yr)')

ax2.legend()

ax2.set_ylim([-10,20])

#ax2.plot(yr,ems1s,'y')
ax2.set_xlim([1990,2300])

ax3.plot(yr,srm1s,'y')
ax3.set_xlim([1990,2300])

ax3.set_ylabel(r'$Wm^{-2}$')

In [ ]:
scens=[ssp126, ssp585, ssp534, ssp460,ssp370,ssp245]

In [ ]:
Cs=[0]*len(scens)
Fs=[0]*len(scens)
Ts=[0]*len(scens)
ss=[0]*len(scens)
es=[0]*len(scens)

Co=[0]*len(scens)
Fo=[0]*len(scens)
To=[0]*len(scens)
so=[0]*len(scens)
eo=[0]*len(scens)

In [ ]:
snames=['SSP126','SSP585','SSP534','SSP460','SSP370','SSP245']

In [ ]:
for i in np.arange(len(scens)):
  Co[i], Fo[i], To[i] = run_fair_clean(scens[i])

In [ ]:
d = {'Start': [2025], 'End': [2175], 'Effic': [1.0], 'fade':[0]}
df = pd.DataFrame(data=d)
Cs[0], Fs[0], Ts[0], ss[0], es[0]=adpt_fair(scens[0],10,1.5,df)



In [ ]:
#SSP585
d = {'Start': [2025], 'End': [2200], 'Effic': [1.0], 'fade':[0]}
df = pd.DataFrame(data=d)
Cs[1], Fs[1], Ts[1], ss[1], es[1]=adpt_fair(scens[1],10,1.5,df)


In [ ]:
d = {'Start': [2025], 'End': [2060], 'Effic': [1.0], 'fade':[0]}
df = pd.DataFrame(data=d)
Cs85_early, Fs85_early, Ts85_early, ss85_early,es85_early=adpt_fair(scens[1],10,1.5,df)


In [ ]:
d = {'Start': [2025], 'End': [2150], 'Effic': [1.0], 'fade':[0]}
df = pd.DataFrame(data=d)
Cs[2], Fs[2], Ts[2], ss[2],es[2]=adpt_fair(scens[2],10,1.5,df)


In [ ]:
#SSP37
d = {'Start': [2030,2060,2100], 'End': [2045,2080,2130], 'Effic': [1.0,1.0,1.0], 'fade':[0,0,0]}
df = pd.DataFrame(data=d)
Cs[4], Fs[4], Ts[4], ss[4],es[4]=adpt_fair(scens[4],10,1.5,df)

In [ ]:
#SSP37
d = {'Start': [2030,2060], 'End': [2045,2080], 'Effic': [1.0,1.0], 'fade':[0,0]}
df = pd.DataFrame(data=d)
Cs37w, Fs37w, Ts37w, ss37w,es37w=adpt_fair(scens[4],10,1.5,df,wd=2080,wf=0.2)

In [ ]:
#SSP46
d = {'Start': [2040,2060], 'End': [2050,2090], 'Effic': [0.5,.6], 'fade':[0,0]}
df = pd.DataFrame(data=d)
Cs[3], Fs[3], Ts[3], ss[3],es[3]=adpt_fair(scens[3],10,1.5,df)

In [ ]:
Ce46, Fe46, Te46 = fair.forward.fair_scm(emissions=scens[3].Emissions.emissions)


In [1]:
#SSP245
d = {'Start': [2035], 'End': [2070], 'Effic': [0.8], 'fade':[30]}
df = pd.DataFrame(data=d)
Cs[5], Fs[5], Ts[5], ss[5],es[5]=adpt_fair(scens[5],10,1.5,df)

NameError: name 'pd' is not defined

In [ ]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
fig = plt.figure(figsize=(15, 8))
ax1=[0]*len(scens)
ax2=[0]*len(scens)
ax3=[0]*len(scens)
for i in np.arange(len(scens)):
  ax1[i] = fig.add_subplot(2,6,i+1)
  ax2[i] = fig.add_subplot(4,6,i+13)
  ax3[i] = fig.add_subplot(4,6,i+19)

  r1=np.random.randn(len(Ts[i]))*0.15
  r2=np.random.randn(len(Ts[i]))*0.15
  ax1[i].plot(yr,Ts[i]+r1,'tab:orange')
  ax1[i].plot(yr,To[i]+r2,'tab:blue',label=snames[i])
  ax1[i].set_xlim([1990,2300])
  ax1[i].legend()
  if i==0:
    ax1[i].set_ylabel('Warming (K)')

  ax2[i].plot(yr,ss[i],'tab:orange')
  ax2[i].set_xlim([1990,2300])
  ax2[i].set_ylim([-10,1])

  if i==0:
    ax2[i].set_ylabel(r'$Wm^{-2}$')

  ax3[i].plot(yr[5:-5],10*np.diff(moving_average(Ts[i]+r1,10)),'tab:orange')
  ax3[i].plot(yr[5:-5],10*np.diff(moving_average(To[i]+r2,10)),'tab:blue')
  ax3[i].set_xlim([1990,2300])
  ax3[i].set_ylim([-3,3])
  if i==0:
    ax3[i].set_ylabel(r'K/decade')
